# Importing libraries

In [19]:
# Basic libraries
import pandas as pd
import time

# Classification models
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB

# Vectorizers
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Utilities and metrics
from itertools import product
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, precision_score, recall_score

# Preprocessing
import nltk
import re

# Download nltk resources
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Setting seeds

In [20]:
s1 = 418380670049
s2 = 5522499
s3 = 29114468376207731

seeds = [s1, s2, s3]

# Importing datasets

In [21]:
path = r'../data/binary/hate/'

train = pd.read_parquet(path + 'train-00000-of-00001.parquet')
test = pd.read_parquet(path + 'test-00000-of-00001.parquet')
val = pd.read_parquet(path + 'validation-00000-of-00001.parquet')

# Dataset preprocessing

In [22]:
stop_words = set(nltk.corpus.stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()

    text = re.sub(r'[^\w\s]', '', text)
    
    words = text.split()
    words = [word for word in words if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words)

train['text'] = train['text'].apply(preprocess_text)

# Text vectorization

In [23]:
tfidf = TfidfVectorizer()
count = CountVectorizer()

vectorizers = [
    TfidfVectorizer(),
    CountVectorizer()
]

# GridSearch implementation

In [24]:
models = {
    'RandomForest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [100, 150, 200],
            'max_depth': [None, 10, 20]
        }
    },
    'SVC': {
        'model': SVC(),
        'params': {
            'C': [1, 10],
            'kernel': ['linear', 'rbf']
        }
    },
    'MultinomialNB': {
        'model': MultinomialNB(),
        'params': {
            'alpha': [0.01, 1.0]
        }
    },
    'LogisticRegression': {
        'model': LogisticRegression(max_iter=1000),
        'params': {
            'C': [1, 10]
        }
    },
    'KNeighbors': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7]
        }
    },
    'DecisionTree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'max_depth': [None, 10, 20]
        }
    },
    'GradientBoosting': {
        'model': GradientBoostingClassifier(),
        'params': {
            'n_estimators': [100, 150, 200],
            'max_depth': [3, 5, 7]
        }
    },
    'AdaBoost': {
        'model': AdaBoostClassifier(),
        'params': {
            'n_estimators': [50, 100, 150]
        }
    },
    'SGD': {
        'model': SGDClassifier(),
        'params': {
            'alpha': [0.0001, 0.001]
        }
    }
}

In [42]:
results = {}

for seed in seeds:
    print(f"Processing seed: {seed}")
    for vectorizer in vectorizers:
        print(f"Processing vectorizer: {vectorizer}")
        for name, info in models.items():
            print(f"Processing model: {name}")
            model = info['model']
            param_grid = info['params']
            param_combinations = product(*param_grid.values())
            
            for combination in param_combinations:
                params = dict(zip(param_grid.keys(), combination))
                model.set_params(**params)

                print(f"Training {name} with params {params} and vectorizer {vectorizer}")
                pipeline = Pipeline([
                    ('vectorizer', vectorizer),
                    ('model', model)
                ])

                start = time.time()
                pipeline.fit(train['text'], train['label'])
                total_time = time.time() - start
                print(f"Training time: {total_time}")

                y_pred = pipeline.predict(val['text'])
                report = classification_report(val['label'], y_pred, output_dict=True)
                
                result_key = (seed, vectorizer, name, tuple(params.items()))
                results[result_key] = {
                    'params': params,
                    'training_time': total_time,
                    'classification_report': report
                }









Processing seed: 418380670049
Processing vectorizer: TfidfVectorizer()
Processing model: RandomForest
Training RandomForest with params {'n_estimators': 100, 'max_depth': None} and vectorizer TfidfVectorizer()
Training time: 16.534723043441772
Training RandomForest with params {'n_estimators': 100, 'max_depth': 10} and vectorizer TfidfVectorizer()
Training time: 0.626702070236206
Training RandomForest with params {'n_estimators': 100, 'max_depth': 20} and vectorizer TfidfVectorizer()
Training time: 1.3966021537780762
Training RandomForest with params {'n_estimators': 150, 'max_depth': None} and vectorizer TfidfVectorizer()
Training time: 23.289532899856567
Training RandomForest with params {'n_estimators': 150, 'max_depth': 10} and vectorizer TfidfVectorizer()
Training time: 0.775524377822876
Training RandomForest with params {'n_estimators': 150, 'max_depth': 20} and vectorizer TfidfVectorizer()
Training time: 1.8080103397369385
Training RandomForest with params {'n_estimators': 200, 

c:\Users\rafae\Documents\TCC\venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Training time: 1.7479774951934814
Training AdaBoost with params {'n_estimators': 100} and vectorizer TfidfVectorizer()


c:\Users\rafae\Documents\TCC\venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Training time: 2.8582088947296143
Training AdaBoost with params {'n_estimators': 150} and vectorizer TfidfVectorizer()


...

Training time: 0.964245080947876
Training AdaBoost with params {'n_estimators': 100} and vectorizer CountVectorizer()


c:\Users\rafae\Documents\TCC\venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Training time: 1.8461363315582275
Training AdaBoost with params {'n_estimators': 150} and vectorizer CountVectorizer()


c:\Users\rafae\Documents\TCC\venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Training time: 2.8862392902374268
Processing model: SGD
Training SGD with params {'alpha': 0.0001} and vectorizer CountVectorizer()
Training time: 0.20819878578186035
Training SGD with params {'alpha': 0.001} and vectorizer CountVectorizer()
Training time: 0.21209049224853516


In [47]:
# convert results to df
results

{(418380670049,
  TfidfVectorizer(),
  'RandomForest',
  (('n_estimators', 100),
   ('max_depth', None))): {'params': {'n_estimators': 100, 'max_depth': None},
  'training_time': 16.534723043441772,
  'classification_report': {'0': {'precision': 0.7223880597014926,
    'recall': 0.8446771378708552,
    'f1-score': 0.7787610619469026,
    'support': 573.0},
   '1': {'precision': 0.7303030303030303,
    'recall': 0.5644028103044496,
    'f1-score': 0.6367239101717305,
    'support': 427.0},
   'accuracy': 0.725,
   'macro avg': {'precision': 0.7263455450022615,
    'recall': 0.7045399740876523,
    'f1-score': 0.7077424860593166,
    'support': 1000.0},
   'weighted avg': {'precision': 0.7257677521483492,
    'recall': 0.725,
    'f1-score': 0.7181111981389041,
    'support': 1000.0}}},
 (418380670049,
  TfidfVectorizer(),
  'RandomForest',
  (('n_estimators', 100), ('max_depth', 10))): {'params': {'n_estimators': 100,
   'max_depth': 10},
  'training_time': 0.626702070236206,
  'classif

# Process results

In [34]:
results.items()

dict_items([((418380670049, TfidfVectorizer(), 'RandomForest', (('n_estimators', 100), ('max_depth', None))), {'params': {'n_estimators': 100, 'max_depth': None}, 'training_time': 17.593130826950073, 'classification_report': {'0': {'precision': 0.7185185185185186, 'recall': 0.8464223385689355, 'f1-score': 0.7772435897435898, 'support': 573.0}, '1': {'precision': 0.7292307692307692, 'recall': 0.5550351288056206, 'f1-score': 0.6303191489361702, 'support': 427.0}, 'accuracy': 0.722, 'macro avg': {'precision': 0.723874643874644, 'recall': 0.7007287336872781, 'f1-score': 0.70378136933988, 'support': 1000.0}, 'weighted avg': {'precision': 0.7230926495726496, 'recall': 0.722, 'f1-score': 0.7145068535188216, 'support': 1000.0}}}), ((418380670049, TfidfVectorizer(), 'RandomForest', (('n_estimators', 100), ('max_depth', 10))), {'params': {'n_estimators': 100, 'max_depth': 10}, 'training_time': 0.7226264476776123, 'classification_report': {'0': {'precision': 0.5916666666666667, 'recall': 0.991273